In [19]:
import alminer, pandas, pymysql, time, glob
import numpy as np

In [20]:
#Query MySQL for SDSS positions
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')
x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47 LIMIT 5000'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
data = np.array(db_data, dtype = float).T
table_id = np.array(db_data, dtype = str).T[0]

In [21]:
ra_sdss = data[1]
dec_sdss = data[2]
z_sdss = data[4]

In [44]:
# split up targets into separate jobs (nJobs)
nJobs = 10

# number of targets in each job
N = int(np.ceil(len(ra_sdss)/nJobs))
print(N)

# number of completed jobs to start from
completed = 3

# run query jobs in serial    
for i in np.arange(completed, nJobs, 1):
    
    ### splitting up targets
    if i == nJobs-1:
        
        mycat = {"Name": table_id[i*N:],
              "RAJ2000" : ra_sdss[i*N:],
              "DEJ2000" : dec_sdss[i*N:]}
        
    else:
    
        mycat = {"Name": table_id[i*N:(i+1)*N],
              "RAJ2000" : ra_sdss[i*N:(i+1)*N],
              "DEJ2000" : dec_sdss[i*N:(i+1)*N]}
    ###
        
    # generate catalogue
    mycat = pandas.DataFrame(mycat)
    
    # run query with ALminer
    start = time.time()
    myquery = alminer.catalog(mycat, search_radius = 0.5, print_targets = False)
    end = time.time()
    print(f'Querying {N} galaxies took {(end-start)/60} min.')
    
    # if there are observations found, print out summary and save for later
    if myquery is None:
        print('No observations found in query.')
    else:
        
        # print out summary
        alminer.summary(myquery)
        
        # pickle data for later
        myquery.to_pickle(f'/arc/projects/salvage/ALMAxmatch/ALminer_output/SDSS-ALMA-search-Job{i}.pkl')


500
alminer.catalog results
Target = 587722982300516634
--------------------------------
No observations found.
--------------------------------
Target = 587722982300451158
--------------------------------
No observations found.
--------------------------------
Target = 587722982300451292
--------------------------------
No observations found.
--------------------------------
Target = 587722982300844287
--------------------------------
No observations found.
--------------------------------
Target = 587722982300778885
--------------------------------
No observations found.
--------------------------------
Target = 587722982300778908
--------------------------------
No observations found.
--------------------------------
Target = 587722982300778927
--------------------------------
No observations found.
--------------------------------
Target = 587722982301040948
--------------------------------
No observations found.
--------------------------------
Target = 587722982300778583
--------

DALFormatError: ConnectTimeout: HTTPSConnectionPool(host='almascience.eso.org', port=443): Max retries exceeded with url: /tap/sync (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fdf6a63f070>, 'Connection to almascience.eso.org timed out. (connect timeout=None)'))

In [45]:
# concatenate queries from Jobs

files = glob.glob('/arc/projects/salvage/ALMAxmatch/ALminer_output/*')

myquery_list = []
for f in files:
    myquery_list.append(pandas.read_pickle(f))
    
myquery = pandas.concat(myquery_list)

In [46]:
alminer.summary(myquery)

--------------------------------
Number of projects = 13
Number of observations = 31
Number of unique subbands = 94
Total number of subbands = 140
25 target(s) with ALMA data = ['1237648704051675158', 'HATLAS1744', 'HATLAS_RED_2207', 'UR7G', 'HATLAS_RED_2221', 'HATLAS_RED_2348', 'GAMA64646', 'HATLASJ121253.5-002203', 'HATLASJ121253', 'HATLAS740', 'HATLAS_RED_1178', 'FJ122021.84-010827.1', 'Callisto', 'HATLAS_J140608.2+005501', 'HATLAS1745', '1237648705664450617', 'GAMA106389', 'HATLAS_RED_2299', 'HATLAS_J143336.7+005042', 'HATLAS_RED_2428', 'HATLASJ121427.3+005819', 'HATLAS1114', 'HATLAS1713', '105386', 'SDSS_J152224.73-004404.5']
--------------------------------


## Match myquery catalog back to SDSS RA/Dec to get redshifts for each observation

In [47]:
ra_query, dec_query = np.array(myquery['RAJ2000']), np.array(myquery['DEJ2000'])

In [48]:
objID_query = []
z_query = []

for ra_q, dec_q in zip(ra_query, dec_query):
    
    diff = np.sqrt((ra_sdss-ra_q)**2 + (dec_sdss-dec_q)**2)
    
    if np.min(diff) > 30:
        print('Poor match!')
    
    objID_query.append(table_id[diff == np.min(diff)][0])
    z_query.append(z_sdss[diff == np.min(diff)][0])

z_query = np.array(z_query)

In the future I can replace the spatial_scale_max with an SDSS radius...

In [49]:
rf_12CO_10 = 115.27120180 #GHz

redshifted_line = rf_12CO_10/(1+z_query)

selected = myquery[(myquery["min_freq_GHz"] < redshifted_line) & 
                   (myquery["max_freq_GHz"] > redshifted_line) & 
                   (myquery["ang_res_arcsec"] < 3) & 
                   (myquery["spatial_scale_max"] > 10) & 
                   ((myquery["scientific_category"] == 'Galaxy evolution') | 
                    (myquery["scientific_category"] == 'Active galaxies') |
                    (myquery["scientific_category"] == 'Cosmology') |
                    (myquery["scientific_category"] == 'Local Universe')) &
                   (myquery["dataproduct_type"] == 'cube')]
alminer.summary(selected)

--------------------------------
Number of projects = 5
Number of observations = 7
Number of unique subbands = 6
Total number of subbands = 9
6 target(s) with ALMA data = ['UR7G', 'HATLASJ121253.5-002203', 'HATLAS_J140608.2+005501', 'GAMA106389', 'HATLAS_J143336.7+005042', '105386']
--------------------------------


In [50]:
rf_12CO_21 = 230.53800000 # 12CO J=2-1
redshifted_line = rf_12CO_21/(1+z_query)

selected = myquery[(myquery["min_freq_GHz"] < redshifted_line) & 
                   (myquery["max_freq_GHz"] > redshifted_line) & 
                   (myquery["ang_res_arcsec"] < 3) & 
                   (myquery["spatial_scale_max"] > 10) & 
                   ((myquery["scientific_category"] == 'Galaxy evolution') | 
                    (myquery["scientific_category"] == 'Active galaxies') |
                    (myquery["scientific_category"] == 'Cosmology') |
                    (myquery["scientific_category"] == 'Local Universe')) &
                   (myquery["dataproduct_type"] == 'cube')]
alminer.summary(selected)

selected = myquery[(myquery["min_freq_GHz"] < redshifted_line) & 
                   (myquery["max_freq_GHz"] > redshifted_line) & 
                   ((myquery["scientific_category"] == 'Galaxy evolution') | 
                    (myquery["scientific_category"] == 'Active galaxies') |
                    (myquery["scientific_category"] == 'Cosmology') |
                    (myquery["scientific_category"] == 'Local Universe')) &
                   (myquery["dataproduct_type"] == 'cube')]
alminer.summary(selected)

--------------------------------
No observations found.
--------------------------------
--------------------------------
Number of projects = 2
Number of observations = 3
Number of unique subbands = 3
Total number of subbands = 3
3 target(s) with ALMA data = ['1237648704051675158', 'GAMA64646', '1237648705664450617']
--------------------------------


In [51]:
rf_12CO_32 = 345.79598990 # 12CO J=3-2
redshifted_line = rf_12CO_32/(1+z_query)

selected = myquery[(myquery["min_freq_GHz"] < redshifted_line) & 
                   (myquery["max_freq_GHz"] > redshifted_line) & 
                   (myquery["ang_res_arcsec"] < 3) & 
                   (myquery["spatial_scale_max"] > 10) & 
                   ((myquery["scientific_category"] == 'Galaxy evolution') | 
                    (myquery["scientific_category"] == 'Active galaxies') |
                    (myquery["scientific_category"] == 'Cosmology') |
                    (myquery["scientific_category"] == 'Local Universe')) &
                   (myquery["dataproduct_type"] == 'cube')]
alminer.summary(selected)

selected = myquery[(myquery["min_freq_GHz"] < redshifted_line) & 
                   (myquery["max_freq_GHz"] > redshifted_line) & 
                   ((myquery["scientific_category"] == 'Galaxy evolution') | 
                    (myquery["scientific_category"] == 'Active galaxies') |
                    (myquery["scientific_category"] == 'Cosmology') |
                    (myquery["scientific_category"] == 'Local Universe')) &
                   (myquery["dataproduct_type"] == 'cube')]
alminer.summary(selected)

--------------------------------
No observations found.
--------------------------------
--------------------------------
Number of projects = 1
Number of observations = 1
Number of unique subbands = 1
Total number of subbands = 1
1 target(s) with ALMA data = ['HATLASJ121253']
--------------------------------


## Function for cedar

In [ ]:
def run_query(ra_sdss, dec_sdss, radius, nJobs, completed = 0)

    # number of targets in each job
    N = int(np.ceil(len(ra_sdss)/nJobs))
    print(N)

    # number of completed jobs to start from
    completed = 3

    # run query jobs in serial    
    for i in np.arange(completed, nJobs, 1):

        ### splitting up targets
        if i == nJobs-1:

            mycat = {"Name": np.arange(completed, nJobs, 1),
                  "RAJ2000" : ra_sdss[i*N:],
                  "DEJ2000" : dec_sdss[i*N:]}

        else:

            mycat = {"Name": np.arange(completed, nJobs, 1),
                  "RAJ2000" : ra_sdss[i*N:(i+1)*N],
                  "DEJ2000" : dec_sdss[i*N:(i+1)*N]}
        ###

        # generate catalogue
        mycat = pandas.DataFrame(mycat)

        # run query with ALminer
        start = time.time()
        myquery = alminer.catalog(mycat, search_radius = radius, print_targets = False)
        end = time.time()
        print(f'Querying {N} galaxies took {(end-start)/60} min.')

        # if there are observations found, print out summary and save for later
        if myquery is None:
            print('No observations found in query.')
        else:

            # print out summary
            alminer.summary(myquery)

            # pickle data for later
            myquery.to_pickle(f'/arc/projects/salvage/ALMAxmatch/ALminer_output/SDSS-ALMA-search-Job{i}.pkl')